In [4]:
import numpy as np
import uproot
%matplotlib inline
from coffea import hist
#from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import cloudpickle
import gzip
import time
import os
import glob
from yahist import Hist1D, Hist2D       

In [3]:
pip install yahist

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached yahist-1.9.0-py3-none-any.whl (26 kB)
  Using cached iminuit-2.4.0-cp38-cp38-manylinux2010_x86_64.whl (312 kB)
  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
Processing /home/jovyan/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56/retrying-1.3.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
class Processor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "") 
        pt_axis = hist.Bin("pt", " [GeV]", [10., 15., 20., 25., 35., 50., 70., 90.])
        eta_mu_axis= hist.Bin("eta", "", [0., 1.2, 2.1, 2.4])
        eta_el_axis= hist.Bin("eta", "", [0., 0.8, 1.479, 2.5])
        
        self._accumulator = processor.dict_accumulator({
            'Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Tight_Muon': hist.Hist("Counts", dataset_axis, pt_axis, eta_mu_axis),
            'Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis),
            'Tight_Electron': hist.Hist("Counts", dataset_axis, pt_axis, eta_el_axis)

        })
    
    @property
    def accumulator(self):
        
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        dataset = events.metadata["dataset"]
        
        # Skim Muon, Electron, and Jet
        Muon = events.Muon
        Muon = Muon[( Muon.pt >=10 ) &
                           (np.abs(Muon.eta) <= 2.4) &
                           (np.abs(Muon.dxy) <= 0.05) &
                           (np.abs(Muon.dz) <= 0.1) &
                           (np.abs(Muon.sip3d) < 4) &
                           (Muon.looseId==1) &
                           (Muon.ptErr/Muon.pt < 0.2) &
                           (Muon.mediumId==1) ]
        
        Electron = events.Electron
        Electron = Electron[( Electron.pt >10 ) &  #  (Electron.isTriggerSafeNolso) 
                           (np.abs(Electron.eta+Electron.deltaEtaSC ) < 2.4) &
                            (Electron.convVeto)&
                            (Electron.lostHits==0) &
                            (Electron.tightCharge==2) &
                            (np.abs(Electron.dz) < 0.1) &
                           (np.abs(Electron.dxy) < 0.05) &
                           (np.abs(Electron.sip3d) < 4) ]
        Jet = events.Jet
        Jet = Jet[( Jet.pt >25 ) &  (np.abs(Jet.eta ) < 2.4) ]

        
        def delta_phi(first, second):
            return np.arccos(np.cos(first.phi - second.phi))

        def delta_r2(first, second):
            return (first.eta - second.eta) ** 2 + delta_phi(first, second) ** 2

        def match(first, second, deltaRCut=0.4):
            drCut2 = deltaRCut**2
            combs = ak.cartesian([first, second], nested=True)
            return ak.any((delta_r2(combs['0'], combs['1'])<drCut2), axis=2)    
        
        def mt(pt1, phi1, pt2, phi2):    #transverse mass
            return np.sqrt( 2*pt1*pt2 * (1 - np.cos(phi1-phi2)) )
        
        # Definition of Tight/Loose Muon and Electron
        Tight_Muon = Muon[( Muon.pt >20 ) &
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.tightId==1) &
                          (Muon.miniPFRelIso_all<0.16)
                         ]
        Tight_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                       #  (Electron.tightId==1) &
                          (Electron.miniPFRelIso_all<0.12)
                         ]
        
        Loose_Muon = Muon[( Muon.pt >20 )&
                          (np.abs(Muon.eta) < 2.4) &
                         (Muon.looseId==1) &
                          (Muon.miniPFRelIso_all<0.4)
                         ]        
        
        Loose_Electron = Electron[( Electron.pt >20 )&
                          (np.abs(Electron.eta) < 2.4) &
                        # (Electron.looseId==1) &
                          (Electron.miniPFRelIso_all<0.4)
                         ]
        MET = events.MET   
        

        control_region_Muon = events[( ak.num(Loose_Muon)==1) &(ak.num(Loose_Electron)==0)
                                    & (ak.num(Jet[~match(Jet, Muon, deltaRCut=1.0)])>=1) & ( MET.pt<20) 
                                    & ak.any(mt(Muon.pt,Muon.phi, MET.pt, MET.phi)<20, axis=1)]

        
        tight_muon= control_region_Muon.Muon[(control_region_Muon.Muon.pt >20 ) &
                          (np.abs(control_region_Muon.Muon.eta) < 2.4) &
                         (control_region_Muon.Muon.tightId==1) &
                          (control_region_Muon.Muon.miniPFRelIso_all<0.16)
                         ]
        output['Tight_Muon'].fill(dataset=dataset, pt=ak.flatten(tight_muon.pt), eta=ak.flatten(tight_muon.eta))
        output['Muon'].fill(dataset=dataset, pt=ak.flatten(control_region_Muon.Muon.pt), eta=ak.flatten(control_region_Muon.Muon.eta))
        return output
    def postprocess(self, accumulator):
        return accumulator

### Getting Samples

The folder QCD containes a file `Samples.txt` with all desired samples, and a script `query.sh`. With the command `./query.sh` (with executing permission), one can generate text files containing all the root file names corresponding to the samples in `Samples.txt` using [dasgoclient](https://github.com/dmwm/dasgoclient).

Note that you need your grid passphrase for setting up the proxy.

In [6]:
Samples = glob.glob("QCD/QCD*.txt")

In [7]:
fileset = {}
for n in Samples:
    name = n.split("/")[1].strip(".txt")
    file = open(n,mode='r')
    content = file.read()
    file.close()
    lst = content.split("\n")
    if lst[-1]=='':
        lst = lst[:-1]
    lst = ["root://xcache/" + s for s in lst]  
    fileset[name]=lst

In [8]:
#fileset = {'SingleMu' : ["root://xcache//store/mc/RunIISummer16NanoAODv7/QCD_Pt-20to30_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8/NANOAODSIM/PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/60000/30AED9F5-92AD-DD46-A547-61765FD6B596.root"]}
#fileset = {'SingleMu' :QCD}

from dask.distributed import Client
client = Client("tls://localhost:8786")

def unique(filename):
    file, ext = os.path.splitext(filename)
    counter = 0
    while os.path.exists(filename):
        counter += 1
        filename = file + str(counter) + ext
    return filename

tstart = time.time()
chunksize = 250000

from dask.distributed import performance_report
fname = unique("dask-report_chunksize=" + str(chunksize/1000) + "K.html")
with performance_report(filename=fname):
    output = processor.run_uproot_job(fileset,
                                     treename='Events',
                                     processor_instance=Processor(),
                                     #executor=processor.futures_executor,
                                     executor=processor.dask_executor,
                                     executor_args={"schema": NanoAODSchema, 'client': client, 'savemetrics': True},
                                     chunksize = chunksize)

In [9]:
dt = time.time() - tstart
outname = 'QCD'
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')
print("Events / s / thread: {:,.0f}".format(output[1]['entries'] / output[1]['processtime']))
print("Events / s: {:,.0f}".format(output[1]['entries'] / dt))

Saving output in histos/QCD.pkl.gz...
Done!
Events / s / thread: 846
Events / s: 69,183


In [10]:
output[1].keys(), output[1]['processtime']

(dict_keys(['bytesread', 'columns', 'entries', 'processtime', 'chunks']),
 875314.5631291866)

### Plotting
https://github.com/cmstas/FTAnalysis/blob/run2/analysis/fakes/derivation/ScanChain_fast.C#L161

In [11]:
import pickle
path = 'histos/QCD.pkl.gz'
print('Opening path: ', path)
hists = {}
with gzip.open(path) as fin:
    QCD= pickle.load(fin)

Opening path:  histos/QCD.pkl.gz


In [12]:
QCD

({'Muon': <Hist (dataset,pt,eta) instance at 0x7f08233d6c10>,
  'Tight_Muon': <Hist (dataset,pt,eta) instance at 0x7f0810797130>,
  'Electron': <Hist (dataset,pt,eta) instance at 0x7f0810797400>,
  'Tight_Electron': <Hist (dataset,pt,eta) instance at 0x7f08107979d0>},
 {'bytesread': 73575139779,
  'columns': {'Electron_convVeto',
   'Electron_deltaEtaSC',
   'Electron_dxy',
   'Electron_dz',
   'Electron_eta',
   'Electron_lostHits',
   'Electron_miniPFRelIso_all',
   'Electron_pt',
   'Electron_sip3d',
   'Electron_tightCharge',
   'Jet_eta',
   'Jet_phi',
   'Jet_pt',
   'MET_phi',
   'MET_pt',
   'Muon_dxy',
   'Muon_dz',
   'Muon_eta',
   'Muon_looseId',
   'Muon_mediumId',
   'Muon_miniPFRelIso_all',
   'Muon_phi',
   'Muon_pt',
   'Muon_ptErr',
   'Muon_sip3d',
   'Muon_tightId',
   'nElectron',
   'nJet',
   'nMuon'},
  'entries': 740240386,
  'processtime': 875314.5631291866,
  'chunks': 3083})

In [32]:
def combine_mat(Dict):
    DICT = {}
    mat = np.zeros((7,3))
    for key, value in Dict.items():
        mat = np.add(mat, value)
    return mat

In [33]:
Loose_Muon = Hist2D.from_bincounts(
    combine_mat(QCD[0]['Muon'].sum().values()).T,
    (QCD[0]['Muon'].axis('pt').edges(), QCD[0]['Muon'].axis('eta').edges())
)

In [34]:
Loose_Muon

[[5379.00 ± 73.34   2521.00 ± 50.21   78063.00 ± 279.40
  46904.00 ± 216.57   15871.00 ± 125.98   5981.00 ± 77.34
  2772.00 ± 52.65]
 [3356.00 ± 57.93   1617.00 ± 40.21   48981.00 ± 221.32
  29698.00 ± 172.33   9928.00 ± 99.64   3511.00 ± 59.25   1510.00 ± 38.86]
 [815.00 ± 28.55   416.00 ± 20.40   12791.00 ± 113.10   7655.00 ± 87.49
  2360.00 ± 48.58   754.00 ± 27.46   285.00 ± 16.88]]

In [35]:
Tight_Muon = Hist2D.from_bincounts(
    combine_mat(QCD[0]['Tight_Muon'].sum().values()).T,
    (QCD[0]['Tight_Muon'].axis('pt').edges(), QCD[0]['Tight_Muon'].axis('eta').edges())
)

In [36]:
Tight_Muon

[[ 0.00 ± 0.00    0.00 ± 0.00   32758.00 ± 180.99   16643.00 ± 129.01
  4492.00 ± 67.02   1578.00 ± 39.72   715.00 ± 26.74]
 [ 0.00 ± 0.00    0.00 ± 0.00   23770.00 ± 154.18   12715.00 ± 112.76
  3555.00 ± 59.62   1167.00 ± 34.16   484.00 ± 22.00]
 [ 0.00 ± 0.00    0.00 ± 0.00   6677.00 ± 81.71   3683.00 ± 60.69
  931.00 ± 30.51   299.00 ± 17.29   106.00 ± 10.30]]

In [37]:
Tight_Muon.divide(Loose_Muon)

[[ 0.00 ± 0.00    0.00 ± 0.00    0.42 ± 0.00    0.35 ± 0.00
   0.28 ± 0.00    0.26 ± 0.01    0.26 ± 0.01]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.49 ± 0.00    0.43 ± 0.00
   0.36 ± 0.01    0.33 ± 0.01    0.32 ± 0.02]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.52 ± 0.01    0.48 ± 0.01
   0.39 ± 0.02    0.40 ± 0.03    0.37 ± 0.04]]